## All the imports

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import dask 
import numpy as np
import xarray as xr
import time
import numpy.ma as ma
import matplotlib.cm as mplcm
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
import os 
import time 
import glob

import cartopy.crs as ccrs
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import matplotlib
matplotlib.rcParams['figure.figsize']= (20, 8)
matplotlib.rcParams['xtick.labelsize']= 22
matplotlib.rcParams['ytick.labelsize']= 22
matplotlib.rcParams['axes.labelsize']= 22
matplotlib.rcParams.update({'font.size': 22})

import zarr
from intake import open_catalog

%matplotlib inline


## Load the data from the catalog

In [ ]:
# AVISO data

cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean.yaml")
ds  = cat["sea_surface_height"].to_dask()
ds

In [ ]:
# eNATL60 ssh

cat2 = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/MEOM-NEMO.yaml")
ds2  = cat2["eNATL60_BLBT02_SSH"].to_dask()
ds2

In [ ]:
# eNATL60 grid

catg = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/MEOM-NEMO.yaml")
dsg  = catg["eNATL60_grid"].to_dask()
dsg

## Request ressources

In [ ]:
from dask.distributed import Client
from dask_gateway import Gateway

gateway = Gateway()
cluster = gateway.new_cluster()
cluster.scale(50)
cluster

In [ ]:
client = Client(cluster)
client

-> click on the link to see the dask workers in action, or copy it and paste it in the dask menu on the left

## Region and dates selection

In [ ]:
#Definition of the Gulf Stream region

box=(-90,-60,30,45)


In [ ]:
#Selection of Gulf Stream region and period of simulation in AVISO data

lonAVISO=ds.longitude.values
where180=np.where(lonAVISO>180)
lonAVISO[where180]=lonAVISO[where180]-360
domainAVISOlon=(box[0]<lonAVISO)*(lonAVISO<box[1])
domainAVISOlat=(box[2]<ds.latitude.values)*(ds.latitude.values<box[3])
whereAVISOlon=np.where(domainAVISOlon)
whereAVISOlat=np.where(domainAVISOlat)

jminAVISO = whereAVISOlat[0][0]
jmaxAVISO = whereAVISOlat[0][-1]
iminAVISO = whereAVISOlon[0][0]
imaxAVISO = whereAVISOlon[0][-1]

aviso_year = ds.sel(time=slice('2009-06-30','2010-10-29'))
myadt = aviso_year.adt[:,jminAVISO:jmaxAVISO+1,iminAVISO:imaxAVISO+1]
lonAVISO_GS=lonAVISO[iminAVISO:imaxAVISO+1]
latAVISO_GS=ds.latitude.values[jminAVISO:jmaxAVISO+1]

In [ ]:
domaineNATL60=(box[0]<dsg.nav_lon)*(dsg.nav_lon<box[1])*(box[2]<dsg.nav_lat)*(dsg.nav_lat<box[3])
whereeNATL60=np.where(domaineNATL60)
jmineNATL60 = whereeNATL60[0].min()
jmaxeNATL60 = whereeNATL60[0].max()
imineNATL60 = whereeNATL60[1].min()
imaxeNATL60 = whereeNATL60[1].max()

ds_sorted=ds2.sortby('time_counter')
ssh_eNATL60=ds_sorted.sossheig
ssh_eNATL60_GS=ssh_eNATL60[:,jmineNATL60:jmaxeNATL60+1,imineNATL60:imaxeNATL60+1]
ssh_eNATL60_1d_2009=ssh_eNATL60.sel(time_counter=slice('2009-06-30','2009-12-31')).groupby('time_counter.dayofyear').mean(dim='time_counter')
ssh_eNATL60_1d_2010=ssh_eNATL60.sel(time_counter=slice('2010-01-01','2010-10-29')).groupby('time_counter.dayofyear').mean(dim='time_counter')
ssh_eNATL60_1d=xr.concat([ssh_eNATL60_1d_2009,ssh_eNATL60_1d_2010],dim='dayofyear')
ssh_eNATL60_1d_GS=ssh_eNATL60_1d[:,jmineNATL60:jmaxeNATL60+1,imineNATL60:imaxeNATL60+1]

loneNATL60_GS=dsg.nav_lon[jmineNATL60:jmaxeNATL60+1,imineNATL60:imaxeNATL60+1]
lateNATL60_GS=dsg.nav_lat[jmineNATL60:jmaxeNATL60+1,imineNATL60:imaxeNATL60+1]

In [ ]:
print(imineNATL60,imaxeNATL60,jmineNATL60,jmaxeNATL60)

## Compute mean and standart deviation 

In [ ]:
# Mean and standart deviation of AVISO data

std_ssh_AVISO_GS = myadt.std(dim='time')
mean_ssh_AVISO_GS = myadt.mean(dim='time') - myadt.mean()


In [ ]:
# Check chunks for eNATL60 ssh
ssh_eNATL60_1d_GS

The data is now chunk along the time dimension dayoftheyear, we rechunk so that the computation of mean and std along this dimension is faster (less dask operations)

In [ ]:
# Rechunk ssh in GS region
ssh_eNATL60_1d_GS_rechunk=ssh_eNATL60_1d_GS.chunk({'dayofyear':487,'x':100,'y':100})

In [ ]:
ssh_eNATL60_1d_GS_rechunk

In [ ]:
# Daily average of hourly ssh for eNATL60 outputs

std_ssh_eNATL60_GS=ssh_eNATL60_1d_GS_rechunk.std(dim='dayofyear')
mean_ssh_eNATL60_GS=ssh_eNATL60_1d_GS_rechunk.mean(dim='dayofyear')-ssh_eNATL60_1d_GS_rechunk.mean()

## Check how many operations are needed to compute

In [ ]:
std_ssh_AVISO_GS

-> 2115 Tasks

In [ ]:
mean_ssh_AVISO_GS

-> 2317 Tasks

In [ ]:
std_ssh_eNATL60_GS

click on the disk icon to see the chunk and number of tasks -> 1 259 513 Tasks !

In [ ]:
mean_ssh_eNATL60_GS

click on the disk icon to see the chunk and number of tasks -> 1 259 927 Tasks !

## Load the data into memory for easier access

In [ ]:
%%time
std_ssh_AVISO_GS.load()

In [ ]:
%%time
mean_ssh_AVISO_GS.load()

In [ ]:
%%time
std_ssh_eNATL60_GS.load()

In [ ]:
%%time
mean_ssh_eNATL60_GS.load()

## Make the plots

In [ ]:
def plot(ax,lat,lon,datamesh,datacont,cmap,extent,vmin,vmax,cont,txt):
    ax.set_extent(extent)
    pcolor=ax.pcolormesh(lon,lat,datamesh,transform=ccrs.PlateCarree(),
                         cmap=cmap,vmin=vmin,vmax=vmax)
    contour=ax.contour(lon,lat,datacont,cont,transform=ccrs.PlateCarree(),
                       colors='k',alpha=0.5,linewidth=0.0001,antialiased=True)
    ax.add_feature(cfeature.LAND,facecolor='grey')
    gl=ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=2, 
                    color='gray', alpha=0.5, linestyle='--')

    fig.subplots_adjust(right=0.8)
    gl.xlocator = mticker.FixedLocator([-80,-85,-70,-65,-60])
    gl.ylocator = mticker.FixedLocator([30,35,40,45,50,55])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 15, 'color': 'black'}
    gl.ylabel_style = {'size': 15, 'color': 'black'}

    cbar = plt.colorbar(pcolor,orientation='vertical',shrink=0.75,label='m')
    ax.text(0.1, 0.7, txt,fontsize=15, color='white', transform=ax.transAxes)


In [ ]:
%%time

fig=plt.figure(figsize=(20,18))

ax1 = fig.add_subplot(211,projection=ccrs.PlateCarree())
plot(ax1,latAVISO_GS,lonAVISO_GS,std_ssh_AVISO_GS,mean_ssh_AVISO_GS,mplcm.YlGnBu,
     [-80,-60,30,45],0,0.5,np.arange(-1,1,0.15),'AVISO')

ax2 = fig.add_subplot(212,projection=ccrs.PlateCarree(central_longitude=-30))
mask=np.squeeze(dsg.tmaskutil[0,jmineNATL60:jmaxeNATL60+1,imineNATL60:imaxeNATL60+1])
data_ma_mesh=np.ma.masked_where(1-mask,std_ssh_eNATL60_GS)
data_ma_cont=np.ma.masked_where(1-mask,mean_ssh_eNATL60_GS)
plot(ax2,lateNATL60_GS,loneNATL60_GS,data_ma_mesh,data_ma_cont,mplcm.YlGnBu,
     [-80,-60,30,45],0,0.5,np.arange(-1,1,0.15),'eNATL60')



In [ ]:
cluster.close()
client.close()